In [21]:
import pandas as pd
import chardet
from bokeh.layouts import row, column
from bokeh.plotting import figure, show
from bokeh.io import output_notebook


In [2]:
def detect_encoding(file_path: str) -> str:
    with open(file_path, 'rb') as file:
        detector = chardet.universaldetector.UniversalDetector()
        for line in file:
            detector.feed(line)
            if detector.done:
                break
        detector.close()
    return detector.result['encoding']

In [3]:
product_id = "com.vansteinengroentjes.apps.ddfive"

In [4]:
import glob
data_path = 'data/'
csv_files = glob.glob(data_path + '*.csv')

# Initialize an empty list to store dataframes
dataframes = []

reviews_df = pd.DataFrame()
sales_df = pd.DataFrame()
crashes_df = pd.DataFrame()
ratings_overview_df = pd.DataFrame()
ratings_country_df = pd.DataFrame()

# encodings = ['utf-8', 'utf-16 LE']

# Loop through the CSV files and read them into dataframes
for file in csv_files:
    # print(file)
    encoding = detect_encoding(file) 
    # for encoding in encodings:
    try:
        df = pd.read_csv(file, encoding=encoding)
        if "reviews" in file:
            # print(file + " " + encoding + " " + str(df.shape))
            reviews_df = pd.concat([reviews_df, df], ignore_index=True)
        elif "sales" in file:
            # print(file + " " + encoding + " " + str(df.shape))
            sales_df = pd.concat([sales_df, df], ignore_index=True)
        elif "crashes" in file:
            # print(file + " " + encoding + " " + str(df.shape))
            crashes_df = pd.concat([crashes_df, df], ignore_index=True)
        elif "ratings" in file:
            if "overview" in file:
                # print(file + " " + encoding + " " + str(df.shape))
                ratings_overview_df = pd.concat([ratings_overview_df, df], ignore_index=True)
            elif "country" in file:
                # print(file + " " + encoding + " " + str(df.shape))
                ratings_country_df = pd.concat([ratings_country_df, df], ignore_index=True)
    except UnicodeDecodeError:
        pass
    

print(len(reviews_df))
print(len(sales_df))
print(len(crashes_df))
print(len(ratings_overview_df))
print(len(ratings_country_df))



78
3487
214
214
18617


# Sales data

In [5]:
# Clean sales data
df = sales_df[sales_df['Product id'] == product_id].copy()
# df["Transaction Date"] = pd.to_datetime(df["Transaction Date"])
# df["Amount (Merchant Currency) Google tax"] = df["Amount (Merchant Currency)"]

def clean_sales_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean and process sales data by combining charge and fee rows.
    
    Args:
        df (pd.DataFrame): Input DataFrame with sales data
        
    Returns:
        pd.DataFrame: Cleaned DataFrame with combined transactions
    """
    # Create a copy of the input DataFrame to avoid the SettingWithCopyWarning
    df = df.copy()
    
    # Convert date and time to datetime, properly handling PDT timezone
    df['datetime'] = pd.to_datetime(
        df['Transaction Date'] + ' ' + df['Transaction Time'].str.replace(' PDT', '')
    )
    
    def combine_transactions(group):
        """Combine charge and fee rows into a single transaction"""
        if len(group) == 2 and set(group['Transaction Type']) == {'Charge', 'Google fee'}:
            charge_row = group[group['Transaction Type'] == 'Charge'].iloc[0]
            fee_row = group[group['Transaction Type'] == 'Google fee'].iloc[0]
            
            # Calculate net amounts
            net_buyer_amount = charge_row['Amount (Buyer Currency)'] + fee_row['Amount (Buyer Currency)']
            net_merchant_amount = charge_row['Amount (Merchant Currency)'] + fee_row['Amount (Merchant Currency)']
            
            # Create result row
            result = charge_row.copy()
            result['Amount (Buyer Currency)'] = net_buyer_amount
            result['Amount (Merchant Currency)'] = net_merchant_amount
            result['Transaction Type'] = 'Net Charge'
            return result
        
        return group.iloc[0]

    # Columns to keep in the final output
    columns_to_keep = [
        'Description', 'datetime', 'Product Title', 
        'Product id', 'Sku Id', 'Buyer Country',
        'Buyer Currency', 'Amount (Buyer Currency)',
        'Merchant Currency', 'Amount (Merchant Currency)'
    ]
    
    # Group and combine transactions
    result_df = (df.groupby('Description', as_index=False)
                  .apply(combine_transactions)
                  [columns_to_keep]
                  .sort_values('datetime')
                  .reset_index(drop=True))
    
    return result_df

# Example usage
cleaned_df = clean_sales_data(df)
print(cleaned_df.head())

                Description            datetime  \
0  GPA.3370-7096-7934-01916 2021-06-01 17:38:09   
1  GPA.3301-2849-0660-49349 2021-06-01 22:46:39   
2  GPA.3372-1497-1097-13226 2021-06-02 07:51:43   
3  GPA.3397-6490-8608-67650 2021-06-02 09:14:16   
4  GPA.3378-4840-7906-77859 2021-06-02 10:32:52   

                                      Product Title  \
0  Character Manager (Complete Reference for DnD 5)   
1           DM Tools (Complete Reference for DnD 5)   
2  Character Manager (Complete Reference for DnD 5)   
3  Character Manager (Complete Reference for DnD 5)   
4           DM Tools (Complete Reference for DnD 5)   

                            Product id                  Sku Id Buyer Country  \
0  com.vansteinengroentjes.apps.ddfive  unlockcharactermanager            US   
1  com.vansteinengroentjes.apps.ddfive                 premium            US   
2  com.vansteinengroentjes.apps.ddfive  unlockcharactermanager            US   
3  com.vansteinengroentjes.apps.ddfive  un

/var/folders/bl/0xp3c78j1gs_14t9fbtq2g3m0000gn/T/ipykernel_69003/1138008851.py:53: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(combine_transactions)


## Sales over time

In [22]:
output_notebook()

def plot_sales_all(df):
    grouped_by_transaction = df.groupby(pd.Grouper(key="datetime", freq='1D'))["Amount (Merchant Currency)"].sum().reset_index()
    grouped_by_count = df.groupby(pd.Grouper(key="datetime", freq='1D')).count().reset_index()
    p = figure(title="Simple line example",x_axis_type="datetime", x_axis_label='date', y_axis_label='sales')
    p.line(grouped_by_transaction["datetime"], grouped_by_transaction["Amount (Merchant Currency)"], legend_label="Sales prices", line_width=2)
    p.line(grouped_by_count["datetime"], grouped_by_count["Amount (Merchant Currency)"], legend_label="Sales volume", line_width=2, color="red")
    # show(p)
    return p

def plot_sales_filtered(df):
    filtered_df = df[df["Sku Id"] == "premium"]
    grouped_by_transaction = filtered_df.groupby(pd.Grouper(key="datetime", freq='1D'))["Amount (Merchant Currency)"].sum().reset_index()
    grouped_by_count = filtered_df.groupby(pd.Grouper(key="datetime", freq='1D')).count().reset_index()
    p = figure(title="Premium",x_axis_type="datetime", x_axis_label='date', y_axis_label='sales')
    p.line(grouped_by_transaction["datetime"], grouped_by_transaction["Amount (Merchant Currency)"], legend_label="Sales prices", line_width=2)
    p.line(grouped_by_count["datetime"], grouped_by_count["Amount (Merchant Currency)"], legend_label="Sales volume", line_width=2, color="red")
    # show(p)
    return p

layout = row(plot_sales_all(cleaned_df.copy()), plot_sales_filtered(cleaned_df.copy()))  # Stacks vertically
show(layout)

Loading BokehJS ...

In [ ]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.layouts import column, row
from bokeh.models import Select, ColumnDataSource, CustomJS

output_notebook()

df = cleaned_df.copy()

def create_daily_metrics(sku_id):
    filtered_df = df[df["Sku Id"] == sku_id]
    sums = filtered_df.groupby(pd.Grouper(key="datetime", freq='1D'))["Amount (Merchant Currency)"].sum().reset_index()
    counts = filtered_df.groupby(pd.Grouper(key="datetime", freq='1D')).count().reset_index()
    return sums, counts

premium_sums, premium_counts = create_daily_metrics("premium")
unlock_sums, unlock_counts = create_daily_metrics("unlockcharactermanager")

# Create ColumnDataSources
premium_source = ColumnDataSource({
    'date': premium_sums['datetime'],
    'sum': premium_sums['Amount (Merchant Currency)'],
    'count': premium_counts['Amount (Merchant Currency)']
})
unlock_source = ColumnDataSource({
    'date': unlock_sums['datetime'],
    'sum': unlock_sums['Amount (Merchant Currency)'],
    'count': unlock_counts['Amount (Merchant Currency)']
})

# Create figure
p = figure(width=800, height=400, x_axis_type="datetime",
          title="Product Revenue Analysis")
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Value'

# Create lines for both metrics
premium_line = p.line('date', 'sum', line_color='blue', 
                     legend_label='Premium', source=premium_source)
unlock_line = p.line('date', 'sum', line_color='red', 
                    legend_label='Character Manager', source=unlock_source)

# Create Select widgets
product_select = Select(title="Product Type", 
                       options=["Both", "Premium Only", "Character Manager Only"],
                       value="Both")

metric_select = Select(title="Metric Type",
                      options=["Revenue", "Count"],
                      value="Revenue")

# Create JavaScript callback
callback = CustomJS(args=dict(premium_line=premium_line,
                            unlock_line=unlock_line,
                            premium_source=premium_source,
                            unlock_source=unlock_source,
                            product_select=product_select,
                            metric_select=metric_select), 
                   code="""
    const product = product_select.value;
    const metric = metric_select.value;
    
    // Update y-values based on metric
    const field = metric === "Revenue" ? "sum" : "count";
    premium_line.glyph.y = {field: field};
    unlock_line.glyph.y = {field: field};
    
    // Update visibility based on product selection
    if (product === "Both") {
        premium_line.visible = true;
        unlock_line.visible = true;
    } else if (product === "Premium Only") {
        premium_line.visible = true;
        unlock_line.visible = false;
    } else {
        premium_line.visible = false;
        unlock_line.visible = true;
    }
""")

# Attach callbacks
product_select.js_on_change('value', callback)
metric_select.js_on_change('value', callback)

# Show the plot
show(column(row(product_select, metric_select), p))

Loading BokehJS ...

# Review data

In [9]:
import pandas as pd
from bokeh.models import Range1d, LinearAxis, RangeTool
from bokeh.layouts import column, row

# Clean ratings data
df_ratings = ratings_country_df.copy()
df_ratings["Date"] = pd.to_datetime(df_ratings["Date"])
df_ratings = df_ratings.groupby(pd.Grouper(key="Date", freq='1D'))["Total Average Rating"].mean()

# Clean crashes data
df_crash = crashes_df.copy()
df_crash["Date"] = pd.to_datetime(df_crash["Date"])
df_crash = df_crash.sort_values(by="Date")

# Normalize data for selection plot
ratings_normalized = (df_ratings - df_ratings.min()) / (df_ratings.max() - df_ratings.min())
crashes_normalized = (df_crash["Daily Crashes"] - df_crash["Daily Crashes"].min()) / (df_crash["Daily Crashes"].max() - df_crash["Daily Crashes"].min())

# create shared range for the x-axis
x_range = (df_ratings.index.min(), df_ratings.index.max())

# Create main plots with original values
p = figure(width=600, height=300, x_axis_type="datetime", x_range=x_range,
          tools="xpan", toolbar_location="above")
p.yaxis.axis_label = 'Rating'
p.line(df_ratings.index, df_ratings.values, line_color='blue')

p2 = figure(width=600, height=300, x_axis_type="datetime", x_range=p.x_range,
           tools="xpan", toolbar_location="above")
p2.yaxis.axis_label = 'Crashes'
p2.line(df_crash["Date"], df_crash["Daily Crashes"], line_color='red')

# Create selection plot with normalized values
select = figure(title="Drag to select time range",
                height=130, width=1200,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None,
                background_fill_color="#efefef")

# Add normalized lines to selection plot
select.line(df_ratings.index, ratings_normalized, line_color='blue')
select.line(df_crash["Date"], crashes_normalized, line_color='red')

# Configure RangeTool
range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "navy"
range_tool.overlay.fill_alpha = 0.2

select.ygrid.grid_line_color = None
select.add_tools(range_tool)

merged_df = pd.merge(df_crash, df_ratings, on='Date', suffixes=('_crash', '_rating'))


dates = pd.date_range(start=x_range[0], end=x_range[1], freq='D')

# from 0 to 1 based on the dates
dates = pd.DataFrame(dates, columns=["Date"])
dates["color"] = (dates["Date"] - dates["Date"].min()) / (dates["Date"].max() - dates["Date"].min())
dates["color"] = (dates["color"] * 255).astype(int)
dates["color"] = dates["color"].apply(lambda x: (x, 150, 150))

# make new plot with crashes vs ratings
p3 = figure(width=1200, height=400, x_axis_label='Crashes', y_axis_label='Rating')
p3.scatter(merged_df["Daily Crashes"], merged_df["Total Average Rating"], color=dates["color"], size=10, fill_alpha=0.6)

# calculate correlation
correlation = merged_df["Daily Crashes"].corr(merged_df["Total Average Rating"])

p3.title.text = f"Correlation: {correlation:.2f} | More red = more recent"


# Show the plots
show(column(row(p, p2), select, p3))

# Spatial data

In [ ]:
df_ratings = ratings_country_df.copy()
df_ratings["Date"] = pd.to_datetime(df_ratings["Date"])
df_ratings = df_ratings.groupby(df_ratings["Country"])["Total Average Rating"].mean()

# get the lat and lon for each country
import geopandas as gpd
lat_long_country = pd.read_csv("./data/longlat.csv")

# Convert latitude and longitude to numeric, coercing errors to NaN
lat_long_country["latitude"] = pd.to_numeric(lat_long_country["latitude"], errors='coerce')
lat_long_country["longitude"] = pd.to_numeric(lat_long_country["longitude"], errors='coerce')

# Drop rows with NaN values
lat_long_country.dropna(subset=["latitude", "longitude"], inplace=True)

lat_long_country.head()

# # convert lat and lon to web mercator
from pyproj import Proj, transform

inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:3857')

lat_long_country["longitude"], lat_long_country["latitude"] = transform(inProj, outProj, lat_long_country["longitude"].values, lat_long_country["latitude"].values)

from bokeh.plotting import figure

p = figure(x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile("CARTODBPOSITRON")

# Show the plot
show(p)




/Users/manuel/Programmeren/School/Assignment 1 attached files 19 February 2025 1622/.venv/lib/python3.13/site-packages/pyproj/crs/crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/manuel/Programmeren/School/Assignment 1 attached files 19 February 2025 1622/.venv/lib/python3.13/site-packages/pyproj/crs/crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/bl/0xp3c78j1gs_14t9fbtq2g3m0000gn/T/ipykernel_69003/402

In [72]:
df_ratings = ratings_country_df.copy()
df_ratings["Date"] = pd.to_datetime(df_ratings["Date"])
df_ratings = df_ratings.groupby(df_ratings["Country"])["Total Average Rating"].mean()
# rename Total Average Rating to rating
df_ratings = df_ratings.rename("rating")

# get the lat and lon for each country
lat_long_country = pd.read_csv("./data/longlat.csv")

# Convert latitude and longitude to numeric, coercing errors to NaN
lat_long_country["latitude"] = pd.to_numeric(lat_long_country["latitude"], errors='coerce')
lat_long_country["longitude"] = pd.to_numeric(lat_long_country["longitude"], errors='coerce')

# Drop rows with NaN values
lat_long_country.dropna(subset=["latitude", "longitude"], inplace=True)

from pyproj import Proj, transform

inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:3857')

lat_long_country["longitude"], lat_long_country["latitude"] = transform(inProj, outProj, lat_long_country["longitude"].values, lat_long_country["latitude"].values)


# merge the dataframes
lat_long_country = pd.merge(lat_long_country, df_ratings, left_on="country", right_index=True)
lat_long_country["color"] = (lat_long_country["rating"] - lat_long_country["rating"].min()) / (lat_long_country["rating"].max() - lat_long_country["rating"].min())
lat_long_country["color"] = (lat_long_country["color"] * 255).astype(int)
lat_long_country["color"] = lat_long_country["color"].apply(lambda x: (255 - x, x, 0))

# amount of start from 0 to 5 using ⭐️ emoji
lat_long_country["stars"] = lat_long_country["rating"].apply(lambda x: "⭐️" * int(round(x)))


print(lat_long_country.head())

from bokeh.models import ColumnDataSource, HoverTool, LabelSet
from bokeh.plotting import figure, show

# Create a ColumnDataSource from your DataFrame
source = ColumnDataSource(lat_long_country)

# Create the figure
p = figure(x_range=(-4000000, 6000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator", width=800, height=600)

# Add the tile
p.add_tile("CARTODBPOSITRON")

# Add circle markers with hover capability
circles = p.circle(x='longitude', y='latitude', size=10, 
                  source=source, fill_color='color', 
                  fill_alpha=0.8, line_color='white')

# Add HoverTool
hover = HoverTool(tooltips=[
    ('Country', '@country'),
    ('Name', '@name'),
    ('Rating', '@rating'),
    ('Stars', '@stars')
])

labels = LabelSet(x='longitude', y='latitude', text='name', x_offset=5, y_offset=5, source=source, border_line_color='black', background_fill_color='white')

p.add_layout(labels)
p.add_tools(hover)

# Show the plot
show(p)

   country      latitude     longitude                    name    rating  \
10      AR -4.638374e+06 -7.081776e+06               Argentina  4.200000   
12      AT  6.026747e+06  1.619707e+06                 Austria  3.834112   
13      AU -2.909486e+06  1.489178e+07               Australia  4.193551   
16      BA  5.452435e+06  1.968026e+06  Bosnia and Herzegovina  5.000000   
18      BD  2.715070e+06  1.005842e+07              Bangladesh  5.000000   

           color       stars  
10  (51, 204, 0)    ⭐️⭐️⭐️⭐️  
12  (75, 180, 0)    ⭐️⭐️⭐️⭐️  
13  (52, 203, 0)    ⭐️⭐️⭐️⭐️  
16   (0, 255, 0)  ⭐️⭐️⭐️⭐️⭐️  
18   (0, 255, 0)  ⭐️⭐️⭐️⭐️⭐️  


/Users/manuel/Programmeren/School/Assignment 1 attached files 19 February 2025 1622/.venv/lib/python3.13/site-packages/pyproj/crs/crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/manuel/Programmeren/School/Assignment 1 attached files 19 February 2025 1622/.venv/lib/python3.13/site-packages/pyproj/crs/crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/bl/0xp3c78j1gs_14t9fbtq2g3m0000gn/T/ipykernel_69003/273